In [56]:
def process(text:str):
    from parse import compile
    from string import punctuation
    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans('', '', punctuation)).lower())
    return " ".join(out_list)


import os
def correct_path(s):
    basename = os.path.basename(s)
    current_dir = "/home/peterr/macocu/task8/transfer_10/"
    return os.path.join(current_dir, basename)
import pandas as pd
df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)

df["path"] = df.path.apply(correct_path)
df["norm_text"] = df.norm_words.apply(" ".join)


dev = df[df.split=="dev"]
test = df[df.split=="test"]


In [57]:
dev.head(2)

,path,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,speaker_info,split,norm_words_edited,norm_text
1,/home/peterr/macocu/task8/transfer_10/Ki_SnDM_...,"18 10 2018 - 9. sjednica, 9. saziv [Ki_SnDM_Ek...",2917.58,2937.50,"[izraelska, tvrtka, prodaje, avione, Hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...","[izraelska, tvrtka, prodaje, avione, hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...",ParlaMint-HR_S09.u4267,ParlaMint-HR_S09.u4267,"{'Speaker_role': 'Regular', 'Speaker_type': 'M...",dev,NaN,izraelska tvrtka prodaje avione hrvatskoj dire...
2,/home/peterr/macocu/task8/transfer_10/XguZsDKd...,"20 1 2017 - 3. sjednica, 9. saziv [XguZsDKdRh4...",13797.59,13811.74,"[jučer, trećem,, sutra, ili, prekosutra, četvr...","[0, 0.43, 1.0, 1.4, 1.52, 2.01, 2.55, 3.12, 3....","[jučer, trećem, sutra, ili, prekosutra, četvrt...","[0, 0.43, 1.0, 1.4, 1.52, 2.01, 2.55, 3.12, 3....",ParlaMint-HR_S03.u9702,ParlaMint-HR_S03.u9702,"{'Speaker_role': 'Regular', 'Speaker_type': 'M...",dev,NaN,jučer trećem sutra ili prekosutra četvrtoj oso...


In [6]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
os.system("cp vocab_300.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/25_normalised_200h/checkpoint-2720").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:

for df in [test, dev]:
    df["predicted"] = df.path.apply(transcribe)

In [47]:
for name, df in zip(["dev", "test"], [dev, test]):
    cer = cer_metric.compute(
        references = df.norm_text,
        predictions = df.predicted
    )
    wer = wer_metric.compute(
        references = df.norm_text,
        predictions = df.predicted
    )

    print(f"split: {name}, {cer=:0.4f}, {wer=:0.4f}")

split: dev, cer=0.0265, wer=0.0877
split: test, cer=0.0234, wer=0.0850


In [48]:
dev.to_csv("30_eval_results_dev.csv", index=False)
test.to_csv("30_eval_results_test.csv", index=False)

# Per speaker wer and cer calculation

In [59]:
df = pd.concat([df, df.speaker_info.apply(pd.Series)], axis=1)
df.head(2)

,path,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,...,norm_words_edited,norm_text,Speaker_role,Speaker_type,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,/home/peterr/macocu/task8/transfer_10/rFVDr4gh...,"20 2 2020 - 1. dio, 16. sjednica, 9. saziv [rF...",10620.07,10632.05,"[interpretacija,, inzistiranje, na, tim, dezin...","[0, 1.0, 1.83, 1.96, 2.22, 3.79, 4.01, 4.43, 5...","[interpretacija, inzistiranje, na, tim, dezinf...","[0, 1.0, 1.83, 1.96, 2.22, 3.79, 4.01, 4.43, 5...",ParlaMint-HR_S16.u4568,ParlaMint-HR_S16.u4568,...,NaN,interpretacija inzistiranje na tim dezinformac...,Regular,MP,HDZ,Klub Hrvatske demokratske zajednice,Coalition,"Tuđman, Miroslav",M,1946
1,/home/peterr/macocu/task8/transfer_10/Ki_SnDM_...,"18 10 2018 - 9. sjednica, 9. saziv [Ki_SnDM_Ek...",2917.58,2937.50,"[izraelska, tvrtka, prodaje, avione, Hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...","[izraelska, tvrtka, prodaje, avione, hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...",ParlaMint-HR_S09.u4267,ParlaMint-HR_S09.u4267,...,NaN,izraelska tvrtka prodaje avione hrvatskoj dire...,Regular,MP,"Živi zid, SNAGA",Klub Živog zida i SNAGA-e,Opposition,"Bunjac, Branimir",M,1972


In [60]:
results = []
for split in ("test", "dev"):
    subset = df[df.split == split]
    for speaker in subset.Speaker_name.unique():
        subsubset = subset[subset.Speaker_name == speaker].copy()
        subsubset["predicted"] = subsubset.path.apply(transcribe)
        speaker_count = subsubset.shape[0]
        cer = cer_metric.compute(
        references = subsubset.norm_text,
        predictions = subsubset.predicted
        )
        wer = wer_metric.compute(
        references = subsubset.norm_text,
        predictions = subsubset.predicted
        )

        print(f"{split=}, {speaker=}, {cer=:0.4f}, {wer=:0.4f}")
        results.append(dict(split=split, speaker=speaker, cer=cer, wer=wer, count=speaker_count))
        

split='test', speaker='Mađerić, Margareta', cer=0.0148, wer=0.0616
split='test', speaker='Bošnjak, Ivica', cer=0.0409, wer=0.1283
split='test', speaker='Kuhar, Maja', cer=0.0247, wer=0.0852
split='test', speaker='Čikotić, Ante', cer=0.0289, wer=0.1052
split='test', speaker='Rimac, Josipa', cer=0.0215, wer=0.0759
split='test', speaker='Grbavac, Damir', cer=0.0162, wer=0.0667
split='dev', speaker='Bunjac, Branimir', cer=0.0262, wer=0.0872
split='dev', speaker='Bačić, Branko', cer=0.0320, wer=0.1099
split='dev', speaker='Mrak-Taritaš, Anka', cer=0.0257, wer=0.0846
split='dev', speaker='Maras, Gordan', cer=0.0233, wer=0.0687
split='dev', speaker='Pernar, Ivan', cer=0.0249, wer=0.0886


In [61]:
per_speaker_df = pd.DataFrame(data=results).sort_values(by="cer", ascending=False)
per_speaker_df

,split,speaker,cer,wer,count
1,test,"Bošnjak, Ivica",0.040908,0.128278,68
7,dev,"Bačić, Branko",0.031961,0.109930,100
3,test,"Čikotić, Ante",0.028933,0.105250,110
6,dev,"Bunjac, Branimir",0.026182,0.087191,100
8,dev,"Mrak-Taritaš, Anka",0.025702,0.084593,100
10,dev,"Pernar, Ivan",0.024897,0.088611,100
2,test,"Kuhar, Maja",0.024664,0.085208,49
9,dev,"Maras, Gordan",0.023275,0.068658,100
4,test,"Rimac, Josipa",0.021518,0.075918,73
5,test,"Grbavac, Damir",0.016243,0.066741,80


In [62]:
per_speaker_df.groupby("split").mean()

,cer,wer,count
split,,,
dev,0.026404,0.087797,100.0
test,0.024519,0.087161,85.5


In [63]:
df[df.split=="train"].shape[0]

380837

In [64]:
free = 183
transfer_10_now = 23
all_samples = 214

i_need = all_samples - transfer_10_now
i_need

191